<a href="https://colab.research.google.com/github/ask404/2018.11-Tutorials/blob/master/DishRAG_v0_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Recipe Dataset**

In [ ]:
!pip install kaggle pandas
from google.colab import files


files.upload()  # Upload kaggle.json
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download crispen5gar/recipes3k
!unzip recipes3k.zip

Saving kaggle.json to kaggle (2).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
Dataset URL: https://www.kaggle.com/datasets/crispen5gar/recipes3k
License(s): GNU Free Documentation License 1.3
recipes3k.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  recipes3k.zip
replace baking.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: baking.json             
  inflating: budget.json             
  inflating: health.json             
  inflating: inspiration.json        
  inflating: recipes.json            


In [ ]:
import pandas as pd
import json

# Load data
df = pd.read_json('recipes.json')  # or pd.read_json()

In [ ]:
df.columns

Index(['id', 'url', 'image', 'name', 'description', 'author', 'rattings',
       'ingredients', 'steps', 'nutrients', 'times', 'serves', 'difficult',
       'vote_count', 'subcategory', 'dish_type', 'maincategory'],
      dtype='object')

In [ ]:
df = df.rename(columns={
    'rattings': 'ratings'
})
df.columns

Index(['id', 'url', 'image', 'name', 'description', 'author', 'ratings',
       'ingredients', 'steps', 'nutrients', 'times', 'serves', 'difficult',
       'vote_count', 'subcategory', 'dish_type', 'maincategory'],
      dtype='object')

In [ ]:
df.head(5)

,id,url,image,name,description,author,ratings,ingredients,steps,nutrients,times,serves,difficult,vote_count,subcategory,dish_type,maincategory
0,7bac6ec3-fb66-4543-80f5-4b692de7a811,https://www.bbcgoodfood.com/recipes/smoked-sal...,https://images.immediate.co.uk/production/vola...,"Smoked salmon, quinoa & dill lunch pot",This easy packed lunch is as delicious as it i...,Chelsie Collins,4,"[2 tbsp half-fat soured cream, 2 tbsp lemon ju...","[First, make the dressing. Mix the soured crea...","{'kcal': '254', 'fat': '7g', 'saturates': '2g'...","{'Preparation': '15 mins', 'Cooking': 'No Time'}",2,Easy,10,Lunch recipes,Quick lunch recipes,recipes
1,25009e0a-0e60-4169-9f76-f464225549a9,https://www.bbcgoodfood.com/recipes/air-fryer-...,https://images.immediate.co.uk/production/vola...,Air fryer chicken breasts,Use an air fryer to create this tempting dish ...,Samuel Goldsmith,5,"[4 chicken breasts, ½ tbsp rapeseed oil, 1 ts...",[Coat the chicken in the oil and set aside. In...,{},"{'Preparation': '5 mins', 'Cooking': '18 mins ...",4,Easy,10,Lunch recipes,Quick lunch recipes,recipes
2,63ff8ff6-2871-4e92-96fd-39964c423a39,https://www.bbcgoodfood.com/recipes/panuozzo-s...,https://images.immediate.co.uk/production/vola...,Panuozzo sandwich,Make your own baguettes and pesto to make thes...,Paul Ainsworth,5,"[300g strong white bread flour, plus extra for...","[Put the flour in a large bowl, then stir in t...",{},"{'Preparation': '20 mins', 'Cooking': '12 mins'}",4,Easy,4,Lunch recipes,Quick lunch recipes,recipes
3,9e62330f-5ea0-4c2a-b0c8-edf386b63fb7,https://www.bbcgoodfood.com/recipes/bulgur-qui...,https://images.immediate.co.uk/production/vola...,Bulgur & quinoa lunch bowls,These meal prep grain bowls use one base and t...,Sara Buenfeld,5,"[1 large onion, very finely chopped, 150g bulg...","[Tip the onion and bulgur mix into a pan, pour...","{'kcal': '369', 'fat': '20g', 'saturates': '4g...","{'Preparation': '5 mins', 'Cooking': '15 mins'}",4,Easy,18,Lunch recipes,Quick lunch recipes,recipes
4,c095701c-a11e-4f47-90a0-a3ea20f7c752,https://www.bbcgoodfood.com/recipes/falafel-bu...,https://images.immediate.co.uk/production/vola...,Falafel burgers,A healthy burger that's filling too. These are...,Good Food team,4,"[400g can chickpeas, rinsed and drained, 1 sm...",[Drain the chickpeas and pat dry with kitchen ...,"{'kcal': '161', 'fat': '8g', 'saturates': '1g'...","{'Preparation': '10 mins', 'Cooking': '6 mins'}",4,Easy,710,Lunch recipes,Quick lunch recipes,recipes


In [ ]:
# If 'tags' doesn't exist, you can create it as an empty list/field
if 'tags' not in df.columns:
    df['tags'] = [[] for _ in range(len(df))]

In [ ]:
import json
# Basic cleaning
def normalize_ingredients(text):
    # Handle both string representations of lists and actual lists:
    if isinstance(text, str):  # Check if 'text' is a string
        try:
            ingredients_list = json.loads(text)  # Try parsing as JSON if it's a string
        except json.JSONDecodeError:
            ingredients_list = eval(text)  # If JSON fails, attempt eval if it's a valid Python expression
    else:
        ingredients_list = text  # If 'text' is already a list, just use it

    return [ing.strip().lower() for ing in ingredients_list]

df['ingredients'] = df['ingredients'].apply(normalize_ingredients)

In [ ]:
# Add vegan tag
def is_vegan(ingredients_list):
    # A simplistic check: if none of certain keywords (like 'meat', 'egg', 'milk') appear
    # This is not comprehensive but just an example:
    non_vegan_keywords = ['meat', 'egg', 'milk', 'chicken',
                          'pork', 'beef', 'fish', 'cheese', 'butter', 'honey']
    for ing in ingredients_list:
        for kw in non_vegan_keywords:
            if kw in ing:
                return False
    return True

df['is_vegan'] = df['ingredients'].apply(is_vegan)
df['tags'] = df.apply(lambda row: row['tags'] + ['vegan'] if row['is_vegan'] else row['tags'], axis=1)




In [ ]:
# Add vegetarian, gluten free, nut free and baby-friendly
diet_categories = {
    'vegetarian': [
        'meat', 'chicken', 'pork', 'beef', 'fish', 'seafood', 'shrimp',
        'lobster', 'bacon', 'ham', 'turkey', 'lamb', 'veal', 'venison',
        'gelatin', 'lard', 'animal broth', 'stock', 'bovine', 'pepsin'
    ],
    'gluten_free': [
        'wheat', 'barley', 'rye', 'malt', 'triticale', 'brewer\'s yeast',
        'spelt', 'farro', 'semolina', 'couscous', 'seitan', 'durum',
        'bulgur', 'wheat flour', 'breadcrumbs', 'pasta', 'graham',
        'matzo', 'emmer', 'einkorn', 'fu', 'udon', 'soy sauce', 'teriyaki'
    ],
    'dairy_free': [
        'milk', 'cheese', 'butter', 'yogurt', 'cream', 'lactose',
        'casein', 'whey', 'ghee', 'curds', 'buttermilk', 'sour cream',
        'ice cream', 'custard', 'pudding', 'lactoglobulin', 'lactalbumin',
        'rennet', 'galactose', 'sodium caseinate', 'condensed milk'
    ],
    'nut_free': [
        'almond', 'cashew', 'walnut', 'pistachio', 'pecan', 'macadamia',
        'hazelnut', 'pine nut', 'brazil nut', 'chestnut', 'peanut',
        'nutella', 'praline', 'marzipan', 'nougat', 'nut oil', 'nut meal',
        'nut extract', 'beechnut', 'gianduja', 'mandelonas', 'nut paste'
    ],
    'baby_friendly': [
        'honey', 'alcohol', 'caffeine', 'raw egg', 'shellfish',
        'shark', 'swordfish', 'tilefish', 'king mackerel', 'added sugar',
        'monosodium glutamate', 'saccharin', 'sucralose', 'aspartame',
        'nitrate', 'nitrite', 'high-fructose', 'artificial color',
        'sodium benzoate', 'sulfites', 'whole nut', 'popcorn',
        'chili pepper', 'whole nuts', 'excess salt', 'coffee', 'tea'
    ]
}

def check_diet(ingredients_list, diet_keywords):
    for ing in ingredients_list:
        for kw in diet_keywords:
            if kw in ing.lower():
                return False
    return True

# Create diet category columns
for diet in diet_categories:
    df[f'is_{diet}'] = df['ingredients'].apply(
        lambda x: check_diet(x, diet_categories[diet])
    )

# Add tags
def update_tags(row):
    for diet in diet_categories:
        if row[f'is_{diet}']:
            row['tags'].append(diet.replace('_', '-'))
    return row

df = df.apply(update_tags, axis=1)

# Optional: Create combined dietary tags column
# df['dietary_tags'] = df.apply(
#     lambda row: [d.replace('_', '-') for d in diet_categories if row[f'is_{d}']],
#     axis=1
# )

In [ ]:
import re
def parse_time_string(time_str):
    """
    Parse time strings like '15 mins', '1 hr 15 mins', 'No Time' into minutes

    Args:
        time_str (str): Time string to parse

    Returns:
        float: Time in minutes, 0 for 'No Time'
    """
    if pd.isna(time_str) or time_str == 'No Time':
        return 0

    # Extract hours and minutes using regex
    hours = re.search(r'(\d+)\s*hr', time_str)
    minutes = re.search(r'(\d+)\s*mins?', time_str)

    total_minutes = 0
    if hours:
        total_minutes += int(hours.group(1)) * 60
    if minutes:
        total_minutes += int(minutes.group(1))

    return total_minutes

def parse_time_column(df, time_col='times'):
    """
    Parse the time dictionary column into separate numerical columns

    Args:
        df (pd.DataFrame): Input DataFrame
        time_col (str): Name of the column containing time dictionaries

    Returns:
        pd.DataFrame: DataFrame with added preparation_mins and cooking_mins columns
    """
    # Create a copy to avoid modifying the original
    df = df.copy()

    # Convert string dictionaries to actual dictionaries if needed
    if isinstance(df[time_col].iloc[0], str):
        df[time_col] = df[time_col].apply(eval)

    # Extract and parse times
    df['preparation_mins'] = df[time_col].apply(
        lambda x: parse_time_string(x.get('Preparation', '0 mins'))
    )
    df['cooking_mins'] = df[time_col].apply(
        lambda x: parse_time_string(x.get('Cooking', '0 mins'))
    )

    # Add total time column
    df['total_time_mins'] = df['preparation_mins'] + df['cooking_mins']

    return df

In [ ]:
df = parse_time_column(df)

In [ ]:
def filter_recipes(df, max_total_time=None, min_rating=None, min_votes=None):
    """
    Filter recipes based on multiple criteria

    Args:
        df (pd.DataFrame): Input DataFrame
        max_total_time (float): Maximum total time in minutes
        min_rating (float): Minimum rating value
        min_votes (int): Minimum number of votes

    Returns:
        pd.DataFrame: Filtered DataFrame
    """
    filtered_df = df.copy()

    if max_total_time is not None:
        filtered_df = filtered_df[filtered_df['total_time_mins'] <= max_total_time]

    if min_rating is not None:
        filtered_df = filtered_df[filtered_df['ratings'] >= min_rating]

    if min_votes is not None:
        filtered_df = filtered_df[filtered_df['vote_count'] >= min_votes]

    return filtered_df


In [ ]:
filtered_df = filter_recipes(
    df,
    max_total_time=60,  # Max 1 hour total time
    min_rating=4.0,     # Minimum 4.0 rating
    min_votes=20        # Minimum 20 votes
)

In [ ]:
filtered_df.head(5)
# Sort by frequency (default is descending)
# filtered_df['dish_type'].value_counts().sort_values(ascending=False)

# Sort by frequency (default is descending)
filtered_df['subcategory'].value_counts().sort_values(ascending=False)


,count
subcategory,
Dinner recipes,50
Lunch recipes,41
Pasta,41
Vegan,39
Breakfast recipes,35
Fish and seafood,32
Storecupboard,31
Vegetarian,31
Desserts,28


In [ ]:
pd.set_option('display.max_colwidth', 400)

In [78]:
df.head(1).transpose()

,0
id,7bac6ec3-fb66-4543-80f5-4b692de7a811
url,https://www.bbcgoodfood.com/recipes/smoked-salmon-quinoa-dill-lunch-pot
image,https://images.immediate.co.uk/production/volatile/sites/30/2020/08/smoked-salmon-quinoa-dill-lunch-pot-0393a04.jpg
name,"Smoked salmon, quinoa & dill lunch pot"
description,"This easy packed lunch is as delicious as it is nutritious, with crunchy cucumber and radishes and a herby, creamy dressing"
author,Chelsie Collins
ratings,4
ingredients,"[2 tbsp half-fat soured cream, 2 tbsp lemon juice, ½ pack dill, finely chopped, 250g pouch ready-to-eat quinoa (we used merchant gourmet), ½ cucumber, halved and sliced, 4 radishes, finely sliced, 100g smoked salmon, torn into strips]"
steps,"[First, make the dressing. Mix the soured cream and lemon juice together in a bowl, then add most of the dill, reserving a quarter for serving., In another bowl, combine the quinoa with the cucumber and radishes, and stir through half the dressing. Season and top with the salmon and the rest of the dill., Put the other half of the dressing in a small pot and drizzle over the quinoa just before..."
nutrients,"{'kcal': '254', 'fat': '7g', 'saturates': '2g', 'carbs': '26g', 'sugars': '3g', 'fibre': '5g', 'protein': '20g', 'salt': '2.5g'}"


In [42]:
filtered_df.iloc[0].to_json()
#filtered_df[0][0]

'{"id":"c095701c-a11e-4f47-90a0-a3ea20f7c752","url":"https:\\/\\/www.bbcgoodfood.com\\/recipes\\/falafel-burgers-0","image":"https:\\/\\/images.immediate.co.uk\\/production\\/volatile\\/sites\\/30\\/2020\\/08\\/recipe-image-legacy-id-326597_11-b7385b9.jpg","name":"Falafel burgers","description":"A healthy burger that\'s filling too. These are great for anyone after a satisfying bite low in calories.","author":"Good Food team","ratings":4,"ingredients":["400g can  chickpeas, rinsed and drained","1 small red onion, roughly chopped","1 garlic clove, chopped","handful of flat-leaf parsley  or curly parsley","1 tsp ground cumin","1 tsp ground coriander","\\u00bd tsp harissa paste  or chilli powder","2 tbsp plain flour","2 tbsp sunflower oil","toasted pitta bread, to serve","200g tub tomato salsa, to serve","green salad, to serve"],"steps":["Drain the chickpeas and pat dry with kitchen paper. Tip into a food processor along with the onion, garlic, parsley, cumin, coriander, harissa paste, fl

**Vector Database Setup**

In [45]:
## Install Pinecone in Colab
!pip install pinecone-client
!pip install openai


In [46]:
from google.colab import userdata

# Embed Each Recipe
import openai
from openai import OpenAI # Import the OpenAI class

openai_key = userdata.get('openai_key')
# Instead of setting openai.api_key, create an OpenAI client instance:
client = OpenAI(api_key=openai_key)

def get_recipe_embedding(text):
    # Use the client to create embeddings:
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    # The API returns a list of embeddings; we take the first one
    embedding = response.data[0].embedding  # Access using .data[0].embedding
    return embedding

# You can combine title, ingredients, and instructions:
def combine_recipe_text(row):
    name = row['name']
    description = row['description']
    ingredients = " ".join(row['ingredients'])  # or keep as comma separated
    steps = row['steps']
    return f"{name}\n\nDescription:\n{description}\n\nIngredients:\n{ingredients}\n\nInstructions:\n{steps}"

filtered_df['combined_text'] = filtered_df.apply(combine_recipe_text, axis=1)

In [47]:
## Compute embeddings for each recipe.
embeddings = []
for idx, row in filtered_df.iterrows():
    text = row['combined_text']
    embedding = get_recipe_embedding(text)
    embeddings.append(embedding)

# Add to the dataframe
filtered_df['embedding'] = embeddings

In [50]:

import pinecone
from pinecone import ServerlessSpec

pinecone_key = userdata.get('pinecone_key')
pinecone_client = pinecone.Pinecone(api_key=pinecone_key)
index_name = "recipe-index"

# Check if index exists
if index_name in pinecone_client.list_indexes().names():
    confirmation = input(f"Warning: Index '{index_name}' already exists. Type 'YES' to delete and recreate: ")
    if confirmation == "YES":
        pinecone_client.delete_index(index_name)
        print(f"Deleted existing index: {index_name}")
    else:
        raise Exception("Operation cancelled by user")

# Create new index
pinecone_client.create_index(
    name=index_name,
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ),
    deletion_protection="disabled"
)
print(f"Created new index: {index_name}")

# Connect to the new index
index = pinecone_client.Index(index_name)

KeyboardInterrupt: Interrupted by user

In [56]:
# Prepare Data for Upsert
index_name = "recipe-index"
# Connect to the new index
index = pinecone_client.Index(index_name)
"""
Pinecone’s upsert expects a list of tuples in the form (id, vector, metadata) where:
- id is a unique string (e.g., the row index or a recipe ID).
- vector is the embedding list (dimension = 1536).
- metadata is a Python dictionary containing any other info you want to store.
"""

data_to_upsert = []
for idx, row in filtered_df.iterrows():
    recipe_id = str(idx)  # or use your own unique ID
    vector = row['embedding']
    metadata = {
        'recipe_id': row['id'],
        'name': row['name'],
        'tags': row['tags'],
        'is_vegan': row['is_vegan'] if 'is_vegan' in row else False,
        'is_vegetarian': row['is_vegetarian'] if 'is_vegetarian' in row else False,
        'is_gluten_free': row['is_gluten_free'] if 'is_gluten_free' in row else False,
        'is_nut_free': row['is_nut_free'] if 'is_nut_free' in row else False,
        'is_dairy_free': row['is_dairy_free'] if 'is_dairy_free' in row else False,
        'is_baby_friendly': row['is_baby_friendly'] if 'is_baby_friendly' in row else False,
        'ratings': row['ratings'],
        'cuisine': row['subcategory'],
        'difficulty': row['difficult']
        # add anything else you want, e.g. cuisine type...
    }
    data_to_upsert.append((recipe_id, vector, metadata))

# Because it's more efficient, upsert in batches
batch_size = 100
for i in range(0, len(data_to_upsert), batch_size):
    batch = data_to_upsert[i:i+batch_size]
    index.upsert(vectors=batch)



In [57]:
# Verify Data
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 390}},
 'total_vector_count': 390}

In [58]:
# Test Similarity Queries: You can use Pinecone’s query endpoint.
test_text = "breakfast dish with eggs cheese carrots and broccoli"
test_vector = get_recipe_embedding(test_text)

In [59]:
result = index.query(vector=test_vector, top_k=5, include_metadata=True)
## Extract ids from index
def extract_recipe_ids(data):
    # Extract recipe_ids from the metadata of each match
    recipe_ids = [match['metadata']['recipe_id'] for match in data['matches']]
    return recipe_ids
id_list = extract_recipe_ids(result)

print(result)

{'matches': [{'id': '53',
              'metadata': {'cuisine': 'Lunch recipes',
                           'difficulty': 'Easy',
                           'is_baby_friendly': True,
                           'is_dairy_free': False,
                           'is_gluten_free': False,
                           'is_nut_free': True,
                           'is_vegan': False,
                           'is_vegetarian': True,
                           'name': 'Easy egg muffins',
                           'ratings': 4.0,
                           'recipe_id': '2f694151-2502-45e8-8637-24a71b707bb9',
                           'tags': ['vegetarian', 'nut-free', 'baby-friendly']},
              'score': 0.844517112,
              'values': []},
             {'id': '200',
              'metadata': {'cuisine': 'Breakfast recipes',
                           'difficulty': 'Easy',
                           'is_baby_friendly': True,
                           'is_dairy_free': False,
       

In [ ]:
import pandas as pd


def display_entries(df, **kwargs):
    """
    Display specific entries in a DataFrame based on various filtering criteria.

    Parameters:
    df (pandas.DataFrame): Input DataFrame
    **kwargs: Filtering criteria
        - columns: List of column names to display
        - row_indices: List of row indices to display
        - conditions: Dict of column:value pairs to filter by
            For single values: {'column': value}
            For list of values: {'column': [value1, value2, ...]}
        - head: Number of first rows to display
        - tail: Number of last rows to display

    Returns:
    pandas.DataFrame: Filtered DataFrame
    """
    result = df.copy()

    # Filter columns if specified
    if 'columns' in kwargs:
        result = result[kwargs['columns']]

    # Filter by row indices
    if 'row_indices' in kwargs:
        result = result.iloc[kwargs['row_indices']]

    # Filter by conditions
    if 'conditions' in kwargs:
        for column, value in kwargs['conditions'].items():
            if isinstance(value, (list, tuple, set)):
                # If value is a list, use isin() for filtering
                result = result[result[column].isin(value)]
            else:
                # For single values, use equality
                result = result[result[column] == value]

    # Get head or tail
    if 'head' in kwargs:
        result = result.head(kwargs['head'])
    elif 'tail' in kwargs:
        result = result.tail(kwargs['tail'])

    return result

In [ ]:
display_entries(filtered_df,
                conditions={'id': id_list }
                #,columns=['name']
                #,'description','steps','subcategory', 'ratings', 'total_time_mins']
                )

CREATE Retrieval Function

This function will:

1.   Take the user’s query (which might include desired ingredients, time constraints, or dietary flags).   
2.   Convert it into a single string to embed.

1.   Optionally parse out dietary requirements (e.g., vegan, baby-friendly) to use as Pinecone filters.

1.   Query Pinecone for the most similar recipes.

1.   Return the top K matching recipes (with metadata and text if needed).





In [68]:
def get_relevant_recipes(user_query, top_k=5):
    """
    user_query: e.g. "vegan dish with chickpeas under 30 minutes, also baby-friendly"
    top_k: number of recipes to retrieve
    """
    # STEP 1: Parse out filters if needed.
    #   This can be fancy NLP or simple keyword checks:
    is_vegan_filter = "vegan" in user_query.lower()
    is_baby_filter = "baby-friendly" in user_query.lower()
    is_vegetarian_filter = "vegetarian" in user_query.lower()
    is_gluten_free_filter = "gluten free" in user_query.lower()
    is_dairy_free_filter = "dairy free" in user_query.lower()
    is_nut_free_filter = "nut free" in user_query.lower()

    # STEP 2: Create a simple text query to embed
    query_embedding = get_recipe_embedding(user_query)

    # STEP 3: Build Pinecone metadata filter
    #   If is_vegan_filter is True, we want is_vegan == True
    #   If is_baby_filter is True, we want is_baby_friendly == True
    #   If not specified, we ignore that filter
    # Pinecone’s filter syntax is a dictionary of "field": value
    pinecone_filter = {}
    if is_vegan_filter:
        pinecone_filter["is_vegan"] = True
    if is_baby_filter:
        pinecone_filter["is_baby_friendly"] = True
    if is_nut_free_filter:
        pinecone_filter["is_nut_free"] = True
    if is_vegetarian_filter:
        pinecone_filter["is_vegetarian"] = True
    if is_gluten_free_filter:
        pinecone_filter["is_gluten_free"] = True
    if is_dairy_free_filter:
        pinecone_filter["is_dairy_free"] = True

    # STEP 4: Query the Pinecone index
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True,
        filter=pinecone_filter if pinecone_filter else None
    )

    # STEP 5: Format the returned matches
    #   Each match has an 'id', 'score', and 'metadata' we can use
    recipes = []
    for match in results['matches']:
        metadata = match['metadata']
        recipe_info = {
            'id': match['id'],
            'score': match['score'],
            'name': metadata.get('name', ''),
            'description': metadata.get('description', ''),
            # 'is_vegan': metadata.get('is_vegan'),
            # 'is_baby_friendly': metadata.get('is_baby_friendly'),
            'ingredients':metadata.get('ingredients',''),
            'steps':metadata.get('steps',''),
            'ratings':metadata.get('ratings',''),
            'total_time_mins':metadata.get('total_time_mins',''),
            'recipe_id': metadata.get('recipe_id')
        }
        recipes.append(recipe_info)

    return recipes

In [69]:
sample_query = "I need a quick vegan recipe for dinner, baby-friendly if possible"
retrieved_recipes = get_relevant_recipes(sample_query, top_k=3)
retrieved_recipes

[{'id': '283',
  'score': 0.827035725,
  'name': 'Black bean, tofu & avocado rice bowl',
  'description': '',
  'ingredients': '',
  'steps': '',
  'ratings': 4.0,
  'total_time_mins': '',
  'recipe_id': '8d52935d-d673-4137-9aad-e3dab576a8d4'},
 {'id': '191',
  'score': 0.825669706,
  'name': 'One-pan spicy rice',
  'description': '',
  'ingredients': '',
  'steps': '',
  'ratings': 5.0,
  'total_time_mins': '',
  'recipe_id': 'b199faa2-eb4f-4ec7-9eb4-f731d06706a1'},
 {'id': '968',
  'score': 0.82538861,
  'name': 'Chickpea curry',
  'description': '',
  'ingredients': '',
  'steps': '',
  'ratings': 5.0,
  'total_time_mins': '',
  'recipe_id': '630e9ac9-5920-4641-86f6-dd57b2bbacea'}]

In [70]:
!pip install firebase-admin

In [76]:
from google.colab import files
import firebase_admin
from firebase_admin import credentials, firestore

# Upload the service account JSON file
uploaded = files.upload()

# Get the filename of the uploaded JSON
filename = list(uploaded.keys())[0]

# Initialize Firebase only once
if not firebase_admin._apps:
    cred = credentials.Certificate(filename)
    firebase_admin.initialize_app(cred)

# Get Firestore client
db = firestore.client()

Saving dishrag-abaa6-firebase-adminsdk-fbsvc-048df1ffdf.json to dishrag-abaa6-firebase-adminsdk-fbsvc-048df1ffdf.json


In [79]:
# Upload Recipes into Firestore

# 1) Convert DF to list of dicts
recipes_list = filtered_df.to_dict(orient='records')

# 2) Upload to Firestore
collection_name = "recipes"
batch_count = 0

for recipe_dict in recipes_list:
    # Use the recipe's "id" as the Firestore doc ID
    doc_id = recipe_dict["id"]
    doc_ref = db.collection(collection_name).document(doc_id)
    doc_ref.set(recipe_dict)
    batch_count += 1

print(f"Uploaded {batch_count} recipes to the '{collection_name}' collection.")

Uploaded 390 recipes to the 'recipes' collection.


In [80]:
## Fetching a Recipe from Firestore
# Create a helper function to fetch recipe data by ID:

def get_recipe_from_db(recipe_id):
    """
    Fetch the recipe document from Firestore by recipe_id.
    Returns None if no document found.
    """
    doc_ref = db.collection("recipes").document(recipe_id)
    doc = doc_ref.get()
    if doc.exists:
        return doc.to_dict()
    else:
        return None

In [99]:
# Revised generate_recipe_suggestion with Firestore

# Below is your updated function. Notice how:

# We only rely on Pinecone to give us a recipe_id.
# We fetch full recipe details from Firestore.
# We build the prompt with the full recipe text, not stored in Pinecone.

import openai

def generate_recipe_suggestion_with_db(retrieved_recipes, user_query):
    """
    Takes top retrieved recipes (with minimal metadata + ID from Pinecone)
    plus user query, fetches full recipe data from Firestore,
    crafts a prompt, calls GPT-3.5, returns the model's response.
    """
    # Build the context from retrieved recipes
    recipe_text_blocks = []

    for i, recipe in enumerate(retrieved_recipes, start=1):
        # retrieved_recipes[i] should contain 'id' (the doc ID in Firestore).
        recipe_id = recipe['id']

        # Fetch full data from Firestore
        full_recipe = get_recipe_from_db(recipe_id)
        if not full_recipe:
            # If no matching doc, skip
            continue

        name = full_recipe.get("name", "Unknown Recipe")
        ingredients = full_recipe.get("ingredients", [])
        steps = full_recipe.get("steps", [])
        description = full_recipe.get("description", "")

        # You can include additional details as needed
        block = (
            f"{i}) {name}\n"
            f"Description: {description}\n"
            f"Ingredients: {', '.join(ingredients)}\n"
            f"Steps: {' '.join(steps)}\n"
        )
        recipe_text_blocks.append(block)

    combined_recipe_text = "\n".join(recipe_text_blocks)

    # Build the prompt
    prompt = f"""
    You are a helpful chef.
    The user wants: {user_query}.

    Here are {len(retrieved_recipes)} recipe options that match:
    {combined_recipe_text}

    Please suggest the best recipe or a new creative combination based on these options.
    Reprint the name, description, author, average user ratings, number of votes, ingredients (including measurements), and steps of the suggested recipe.
    Explain why it's a good fit, and keep it concise.
    Only use the recipes provided for your answer.
    At the end print the name, description, author, average user ratings, and number of votes of two other top ranked recipes and ask if the user would like to see the full recipe of either.
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful recipe assistant."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.6,
        max_tokens=650
    )
    # Access the content using .choices[0].message.content
    answer = response.choices[0].message.content
    return answer

In [100]:
sample_query = "I need a quick vegan recipe for dinner, baby-friendly if possible"
retrieved_recipes = get_relevant_recipes(sample_query, top_k=3)
retrieved_recipes
# Example usage (assuming you already have retrieved_recipes from Pinecone):
test_answer = generate_recipe_suggestion_with_db(retrieved_recipes, sample_query)
print(test_answer)

Based on your request for a quick vegan recipe that is also baby-friendly, I recommend the following recipe:

### Recipe: Creamy Avocado Pasta

**Description:** A quick and creamy pasta dish made with ripe avocados and fresh herbs, perfect for a nutritious and easy dinner.

**Author:** Vegan Cooking Guru

**Average User Ratings:** 4.8/5

**Number of Votes:** 150

**Ingredients:**
- 8 oz whole wheat pasta (spaghetti or your choice)
- 2 ripe avocados
- 1 clove garlic
- 2 tablespoons lemon juice
- 1/4 cup fresh basil leaves
- Salt and pepper to taste
- Optional: cherry tomatoes for garnish

**Steps:**
1. Cook the pasta according to package instructions until al dente. Drain and set aside.
2. In a blender or food processor, combine the avocados, garlic, lemon juice, basil, salt, and pepper. Blend until smooth and creamy.
3. Toss the cooked pasta with the avocado sauce until well coated.
4. Serve immediately, garnished with cherry tomatoes if desired.

**Why it's a good fit:** This recipe i

Gradio

In [ ]:
#import Gradio
!pip install gradio
import gradio as gr
